<a href="https://colab.research.google.com/github/ashikshafi08/Learning_Tensorflow/blob/main/Notebooks/04_Transfer_Learning_Feature_Extraction_with_TensorFlow.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!nvidia-smi

Fri Mar 19 13:09:16 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.56       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   42C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Transfer Learning with TensorFlow - Feature Extraction

### What is Transfer Learning
Surely someone has spend the time crafting the right model for your job...
- We can leverage an **existing neural network architecture** which is proven to work on problems similar to our own. 
- Can leverage a working network architecture which has **already learned  patterns similar data** to our own. (often results in great results with less data)

Transfer Learning is leveraging a working model's existing architecture and learned patterns for our own problem.

### Things will be doing in this lecture : 
- Introduce Transfer Learning with TensorFlow. 
- Using a small dataset to experiment faster (10% training examples). 
- Building a **transfer learning feature extraction** model with TensorFlow Hub.
- Use TensorBoard to track modelling experiments and results.

## Download and Becoming one with the Data

In [2]:
# Get data  (10% of 10 food classes From Food101)
import zipfile

# Download the data 
!wget https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip

# Unzip the downloaded file
zip_ref = zipfile.ZipFile('10_food_classes_10_percent.zip')
zip_ref.extractall()
zip_ref.close()

--2021-03-19 13:21:33--  https://storage.googleapis.com/ztm_tf_course/food_vision/10_food_classes_10_percent.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.20.128, 74.125.197.128, 74.125.142.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.20.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 168546183 (161M) [application/zip]
Saving to: ‘10_food_classes_10_percent.zip’

10_food_classes_10_ 100%[===================>] 160.74M   176MB/s    in 0.9s    

2021-03-19 13:21:34 (176 MB/s) - ‘10_food_classes_10_percent.zip’ saved [168546183/168546183]



In [5]:
# How many images in each folder? 
import os 

# Walkthrough 10 percent data directory and list number of files
for dirpath , dirnames , filenames in os.walk('10_food_classes_10_percent'):
  print(f'There are {len(dirnames)} directories and {len(filenames)} images in "{dirpath}"')

There are 2 directories and 0 images in "10_food_classes_10_percent"
There are 10 directories and 0 images in "10_food_classes_10_percent/test"
There are 0 directories and 250 images in "10_food_classes_10_percent/test/steak"
There are 0 directories and 250 images in "10_food_classes_10_percent/test/chicken_wings"
There are 0 directories and 250 images in "10_food_classes_10_percent/test/chicken_curry"
There are 0 directories and 250 images in "10_food_classes_10_percent/test/ice_cream"
There are 0 directories and 250 images in "10_food_classes_10_percent/test/hamburger"
There are 0 directories and 250 images in "10_food_classes_10_percent/test/ramen"
There are 0 directories and 250 images in "10_food_classes_10_percent/test/grilled_salmon"
There are 0 directories and 250 images in "10_food_classes_10_percent/test/fried_rice"
There are 0 directories and 250 images in "10_food_classes_10_percent/test/pizza"
There are 0 directories and 250 images in "10_food_classes_10_percent/test/sushi

Breaking down the train and test set, 
- `train_set` --> has 75 images in each class. 
- `test_set` --> has 200 images in each class.

## Create some dataloaders (preparing the data)

We'll use `ImageDataGenerator` class to load in our images in batches. 

In [16]:
# Setup data inputs 
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Setting up global variable (Variable in CAPS --> means some sort of hyperparameters)
IMG_SHAPE = (224 , 224)
BATCH_SIZE = 32

# Setting up the path to train and test
train_dir = '10_food_classes_10_percent/train/'
test_dir = '10_food_classes_10_percent/test/'

# Creating a instance of the ImageDataGenerator 
train_datagen = ImageDataGenerator(rescale = 1/255.)
test_datagen = ImageDataGenerator(rescale= 1/255.)

# Applying the generator to rescale our data
print(f'Training Images......')
train_data_10_percent = train_datagen.flow_from_directory(train_dir , 
                                                          target_size = IMG_SHAPE, 
                                                          batch_size = BATCH_SIZE , 
                                                          class_mode = 'categorical')

print('\nTesting Images.....')
test_data_10_percent = test_datagen.flow_from_directory(test_dir , 
                                                        target_size = IMG_SHAPE , 
                                                        batch_size = BATCH_SIZE , 
                                                        class_mode = 'categorical')

Training Images......
Found 750 images belonging to 10 classes.

Testing Images.....
Found 2500 images belonging to 10 classes.


## Setting up Callbacks (things to run whilst our model trains)

Callbacks are extra-functionality you can add to your models to be performed during or after training. Some of the most popular callbacks are: 
- Tracking Experiments with `TensorBoard` callback. 
- Model checkpoint with `ModelCheckpoint` callback.
- Stopping a model from training (before it trains too long and overfits) with the `EarlyStopping` callback. 

Create TensorBoard Callback, now we want to functionized because we need to create a new folder for each model.  

In [26]:
# Creating TensorBoard 
import datetime # timestamps for when our model has run

def create_tensorboard_callback(dir_name , experiment_name):

  # Creating the log directory to save the experiments
  log_dir = dir_names + '/' + experiment_name + '/' + datetime.datetime.now().strftime("%Y/%m/%d - %H:%M:%S")

  # Creating a tensorboard
  tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir = log_dir)
  print(f'Saving tensorboard log files to: {log_dir}')
  return tensorboard_callback

## Creating models using TensorFlow Hub

In the past we've used TensorFlow to create our own models layer by layer from scratch. 

Now we're going to do a similar process, except the majority of our model's layers are going to come from [TensorFlow Hub](https://tfhub.dev/)

Browsing the TensorFlow Hub page and sorting for image classification, we found the following feature vector model link https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1

Often in Deeplearning, higher the number efficientnet(b0......b7) means the more complex the model. 

- Resnet : https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/4

 

In [27]:
# Let's compare the following two models
resnet_url = 'https://tfhub.dev/google/imagenet/resnet_v2_50/feature_vector/4'
efficientnet_url = 'https://tfhub.dev/tensorflow/efficientnet/b0/feature-vector/1'

In [28]:
# Import dependencies 
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow.keras import layers

In [ ]:
# Let create function create_model() to create a model from URL
def create_model(model_url , num_classes = 10):
  '''
  Takes a TensorFlow Hub URL and creates a Keras Sequential model with it

  Args: 
     model_url (str): A TensorFlow Hub feature extraction URL 
     num_classes (int): Number of output neuron in the output layer, should be 
                       equal to the number of target classes, default 10. 

  Returns: 
  An uncompiled Keras Sequential model with model_url as feature extractor layer 
  and Dense output layer with num_classes output neurons. 
  '''
  # Download the pretrained model and save it as a Keras layer
  feature_extractor_layer = hub.KerasLayer(model_url , 
                                           trainable = False) # freeze the already learnt model 

